# Puntuacion de crédito

# Introduccion

En este proyecto se lleva a cabo la evaluación de los clientes con respecto a la solvencia crediticia la cual es fundamental para determinar tanto la capacidad de pago como la reduccion los riesgos de impuntualidades en los pagos. El objetivo principal será investigar si variables como el estado civil, el número de hijos, propositos e ingresos de un cliente tienen algún impacto en el incumplimiento de un préstamo

## 1.- Abrir archivo de datos

In [1]:
#librerias
import pandas as pd

In [2]:
#abrir el archivo
df = pd.read_csv('/datasets/credit_scoring_eng.csv')

## 2.-Preprocesamiento de datos

In [3]:
#informacion general
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
#numero de datos ausentes
print(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [5]:
missing_data = df[['days_employed', 'total_income']].isnull().sum()
symmetry = missing_data[0] == missing_data[1]

print(f"¿Los valores nulos son simétricos? {symmetry}")

¿Los valores nulos son simétricos? True


In [6]:
#observaciones sobre patrones en valores nulos
df_no_null = df.dropna()
df_null_values = df[df['days_employed'].isnull() | df['total_income'].isnull()]
for column in ['days_employed', 'total_income']:
    print(f"Distribución de respuestas en {column}:")
    print("df_no_null:")
    print(df_no_null[column].value_counts(normalize=True))
    print("df_null_values:")
    print(df_null_values[column].value_counts(normalize=True))
    print()

Distribución de respuestas en days_employed:
df_no_null:
-327.685916     0.000052
-1580.622577    0.000052
-4122.460569    0.000052
-2828.237691    0.000052
-2636.090517    0.000052
                  ...   
-7120.517564    0.000052
-2146.884040    0.000052
-881.454684     0.000052
-794.666350     0.000052
-3382.113891    0.000052
Name: days_employed, Length: 19351, dtype: float64
df_null_values:
Series([], Name: days_employed, dtype: float64)

Distribución de respuestas en total_income:
df_no_null:
42413.096    0.000103
17312.717    0.000103
31791.384    0.000103
14427.878    0.000052
20837.034    0.000052
               ...   
27715.458    0.000052
23834.534    0.000052
26124.613    0.000052
28692.182    0.000052
41428.916    0.000052
Name: total_income, Length: 19348, dtype: float64
df_null_values:
Series([], Name: total_income, dtype: float64)



Efectivamente los valores nulos en days_employed y total_income son simetricos, pues existe la misma cantidad de valores ausentes en ambas columnas


In [7]:
#correccion de education (tiene un mal uso de mayúsculas y minúsculas)
df['education'] = df['education'].str.lower()
print(df.head())

   children  days_employed  dob_years            education  education_id  \
0         1   -8437.673028         42    bachelor's degree             0   
1         1   -4024.803754         36  secondary education             1   
2         0   -5623.422610         33  secondary education             1   
3         3   -4124.747207         32  secondary education             1   
4         0  340266.072047         53  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1            married                 0      F    employee     0     17932.802   
2            married                 0      M    employee     0     23341.752   
3            married                 0      M    employee     0     42820.568   
4  civil partnership                 1      F     retiree     0     25378.572   

                   purpose  
0    purchase of the house 

In [8]:
#correccion de children (hay valores inconsistentes como -1 y 20)
unique_children = df['children'].unique()
print(unique_children)

df['children']=df['children'].abs()
df['children']=df['children'].replace(20, 2)

unique_children = df['children'].unique()
print(unique_children)

[ 1  0  3  2 -1  4 20  5]
[1 0 3 2 4 5]


In [9]:
# Corrección de days_employed (tiene números negativos y también anormalmente grandes)
df['days_employed'] = df['days_employed'].apply(lambda x: -x if x < 0 else x)

max_age = df['dob_years'].max()
df['days_employed'] = df['days_employed'].apply(lambda x: x if x < max_age * 365 else x / 365)

# Verificación de los valores corregidos
unique_days_employed = df['days_employed'].unique()
print(unique_days_employed)


[8437.67302776 4024.80375385 5623.42261023 ... 2113.3468877  3112.4817052
 1984.50758853]


Se corrigieron datos negativos y anormales

In [10]:
#correccion de dob_years (tiene 0 s)
mean = df['dob_years'].mean()
df['dob_years'] = df['dob_years'].replace(0, mean)

unique_dob = df['dob_years'].unique()
print(unique_dob)

[42.         36.         33.         32.         53.         27.
 43.         50.         35.         41.         40.         65.
 54.         56.         26.         48.         24.         21.
 57.         67.         28.         63.         62.         47.
 34.         68.         25.         31.         30.         20.
 49.         37.         45.         61.         64.         44.
 52.         46.         23.         38.         39.         51.
 43.29337979 59.         29.         60.         55.         58.
 71.         22.         73.         66.         69.         19.
 72.         70.         74.         75.        ]


In [11]:
#correccion de gender (hay que identificar a que se puede referir XNA).
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [12]:
#correccion de familly_status (su contenido debe estar en snake_case).
df['family_status'] = df['family_status'].str.lower().str.replace(' ', '_')

In [13]:
#correccion de income_type (el contenido debe estar en snake_case y las categorías de menor frecuencia pueden seer agrupadas)
df['income_type'] = df['income_type'].str.lower().str.replace(' ', '_')

In [14]:
# Crear dos variables de agrupamientos (age_grouping y income_grouping)
df['age_grouping'] = pd.cut(df['dob_years'], bins=[0, 30, 40, 50, 100], labels=['<30', '30-40', '40-50', '>50'])
df['income_grouping'] = pd.qcut(df['total_income'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

In [15]:
#llena datos ausentes en days_employed y total_income
df['days_employed'].fillna(df.groupby('age_grouping')['days_employed'].transform('median'), inplace=True)
df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'), inplace=True)

In [16]:
#verificacion de datos ausentes
print(df.isna().sum())

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
age_grouping           0
income_grouping     2174
dtype: int64


In [17]:
#imprime las primeras 10 filas para reemplazar los numeros reales con numeros enteros
print(df.head(10))

   children  days_employed  dob_years            education  education_id  \
0         1    8437.673028       42.0    bachelor's degree             0   
1         1    4024.803754       36.0  secondary education             1   
2         0    5623.422610       33.0  secondary education             1   
3         3    4124.747207       32.0  secondary education             1   
4         0     932.235814       53.0  secondary education             1   
5         0     926.185831       27.0    bachelor's degree             0   
6         0    2879.202052       43.0    bachelor's degree             0   
7         0     152.779569       50.0  secondary education             1   
8         2    6929.865299       35.0    bachelor's degree             0   
9         0    2188.756445       41.0  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1

In [18]:
#reemplaza numero real con numero entero
df['days_employed']= df['days_employed'].astype('int')
df['total_income']= df['total_income'].astype('int')
df['children']= df['children'].astype('int')
df['days_employed']= df['days_employed'].astype('int')
df['dob_years']= df['dob_years'].astype('int')
df['education_id']= df['education_id'].astype('int')
df['family_status_id']= df['family_status_id'].astype('int')
df['debt']= df['debt'].astype('int')
df['children']= df['children'].astype('int')


In [19]:
#verifica mediante info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   children          21525 non-null  int64   
 1   days_employed     21525 non-null  int64   
 2   dob_years         21525 non-null  int64   
 3   education         21525 non-null  object  
 4   education_id      21525 non-null  int64   
 5   family_status     21525 non-null  object  
 6   family_status_id  21525 non-null  int64   
 7   gender            21525 non-null  object  
 8   income_type       21525 non-null  object  
 9   debt              21525 non-null  int64   
 10  total_income      21525 non-null  int64   
 11  purpose           21525 non-null  object  
 12  age_grouping      21525 non-null  category
 13  income_grouping   19351 non-null  category
dtypes: category(2), int64(7), object(5)
memory usage: 2.0+ MB


In [20]:
#encuentra el numero de duplicados, los elimina y verifica
print(df.duplicated().sum())
df= df.drop_duplicates().reset_index(drop=True)
print(df.duplicated().sum())


71
0


In [21]:
# Clasifica los datos por distintas variables
df['purpose_classification'] = df['purpose'].str.lower()
df['purpose_classification'] = df['purpose_classification'].apply(lambda x: 'real_estate' if 'real' in x else x)
df['purpose_classification'] = df['purpose_classification'].apply(lambda x: 'car' if 'car' in x else x)
df['purpose_classification'] = df['purpose_classification'].apply(lambda x: 'wedding' if 'wedding' in x else x)
df['purpose_classification'] = df['purpose_classification'].apply(lambda x: 'university' if 'education' in x else x)


In [22]:
def cat_purpose(row):
    purpose = row['purpose']
    if 'hous' in purpose or 'estate' in purpose or 'home' in purpose:
        return 'real_estate'
    elif 'car' in purpose:
        return 'car'
    elif 'wedding' in purpose:
        return 'wedding'
    else:
        return 'university'

df['cat_purpose'] = df.apply(cat_purpose, axis=1)

Listo

## Preguntas clave

¿Hay alguna conexión entre tener hijos y pagar un préstamo a tiempo?
Las personas con mas hijos tardan mas en pagar

In [23]:
df.pivot_table(index='children', values='debt', aggfunc='mean')

,debt
children,
0,0.075438
1,0.091658
2,0.094925
3,0.081818
4,0.097561
5,0.000000


En general, parece haber una ligera conexión entre tener hijos y el incumplimiento de pago de un préstamo o su pago de forma impuntual, pues los cumplimientos aumentan ligeramente a medida que aumenta el número de hijos

¿Existe una conexión entre el estado civil y el pago a tiempo de un préstamo?
Las personas casadas son mas puntuales

In [24]:
df.pivot_table(index='family_status', values='debt', aggfunc='mean')

,debt
family_status,
civil_partnership,0.093471
divorced,0.071130
married,0.075452
unmarried,0.097509
widow_/_widower,0.065693


Se puede ver que las personas casadas y viudas tienen un menor ratio de incumplimiento de pago en comparación con otros estados civiles, pudiera ser por el apoyo que se tiene al estar en matrimonio

¿Existe una conexión entre el nivel de ingresos y el pago a tiempo de un préstamo?
Mientras mas alto el ingreso mas puntual

In [25]:
# Creamos una función para categorizar los ingresos
def cat_income(value):
  if value < 17000:
    return 'low'
  elif value < 23000:
    return 'middle'
  elif value < 32000:
    return 'middle-high'
  else:
    return 'high'

# Ahora implementamos la función
df['cat_income'] = df['total_income'].apply(cat_income)


¿Cómo afectan los diferentes propósitos del préstamo al reembolso a tiempo del préstamo?
Los pagos son mucho mas puntuales

Los diferentes propósitos pueden influir en el reembolso puntual. Los préstamos destinados a las propiedades parecen tener tasas de pago a tiempo más altas, y los préstamos relacionados con la educación y autos tienden a tener tasas de incumplimiento de pago más altas.

¿Existe una conexión entre el nivel de ingresos y el pago a tiempo de un préstamo?
Sí, definitivamente existe una conexión entre el nivel de ingresos y el pago a tiempo de un préstamo
Las personas con  ingresos altos tienen una mayor capacidad para pagar, mientas que los ingresos bajos pueden enfrentar más dificultades para cumplir con los pagos

In [28]:
df.pivot_table(index='cat_income', values='debt', aggfunc='mean')

,debt
cat_income,
high,0.070653
low,0.078398
middle,0.089614
middle-high,0.084627


¿Cómo afectan los diferentes propósitos del préstamo al reembolso a tiempo del préstamo?
Los diferentes propósitos del préstamo pueden afectar el reembolso a tiempo del préstamo de diferentes maneras, los autos suelen tener mucho mas dificultades a la hora de pagar, mientras que las propiedades son mas puntuales, las bodas igualmente tienen buena capacidad de pago y las universidades pudieran ser un poco mas impuntuales

In [27]:
df.pivot_table(index='cat_purpose', values='debt', aggfunc='mean')

,debt
cat_purpose,
car,0.093590
real_estate,0.071558
university,0.085483
wedding,0.080034


## Conclusión


En conclusión, el análisis muestra ciertas conexiones entre alguna variables como tener hijos, estado civil, nivel de ingresos y propósito del préstamo, y el pago a tiempo del préstamo.

Tener hijos:Podemos ver que las personas con más hijos tienden a tener una tasa ligeramente más alta de incumplimiento en el pago de préstamos.

Estado civil: Las personas casadas tienen una tasa de pago a tiempo ligeramente más alta en comparación con las personas solteras, divorciadas o viudas. Esto puede deberse a una mayor estabilidad financiera y apoyo mutuo en las parejas casadas.

Nivel de ingresos: los datos no parecian tener una tendencia hacia hcia personas con mayor ni menor ingreso, ambas partes tenian distintas tasas de incumplimineto

Propósito del préstamo: Los préstamos destinados a la compra de viviendas tienen tasas de pago a tiempo más altas, mientras que los préstamos relacionados con la educación y la compra de automóviles tienden a tener tasas de incumplimiento de pago más altas